# 콜백

사용자의 입력에 따라 변하는 어플리케이션을 개발해야하는 경우가 있다. 입력에서 발생하는 변화를 감지해 호출되는 콜백 함수를 지원한다.

 - https://dash.plotly.com/basic-callbacks

In [2]:
import pandas as pd

In [3]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

Dash 콜백은 어플리케이션의 "입력"및 "출력" 에 사용자 이벤트를 처리할 함수에 `@app.callback` 데코레이터로 선언적으로 표현된다. 

```python
@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)

```

아래 샘플 코드는 Dash에게 "Input"(입력) 컴포넌트의 값(value)이 변할 때마다 "Output"(출력) 컴포넌트의 chlildren을 업데이트하라고 알려주는 것이다.

- `@app.callback` 데코레이터로 래핑된 함수의 이름은 사용자가 원하는 대로 지정할 수 있다. 컨벤션은 콜백 아웃풋을 나타내는 함수명을 사용하는 것이다.
- 함수 인수도 어떤 이름이든 사용할 수 있다. 하지만 인수의 순서는 맞춰줘야 한다.(데코레이터와 동일한 순서로)
- `@app.callback` 데코레이터의 입력 또는 출력으로 참조 할 때 app.layout에서 Dash 컴포넌트에 지정한 것과 동일한 ID를 사용해야 한다.
- `@app.callback` 데코레이터는 콜백 함수 선언 바로 위에 있어야합니다. 데코레이터와 함수 정의 사이에 빈 줄이 있으면 콜백 등록이 되지 않는다.

In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = Dash(__name__)
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div(["Input: ",
              dcc.Input(id='my-input', value='initial value', type='text')]),
    html.Br(),
    html.Div(id='my-output'),
])


@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

Dash is running on http://127.0.0.1:8050/



### 슬라이더 처리 콜백

dcc.Slidera가 a를 업데이트하는 또 다른 예를 살펴보겠습니다 dcc.Graph.

 -  update_figure이 함수는 이 새 값으로 데이터 프레임을 필터링하고 figure객체를 생성한 다음 대시 애플리케이션에 반환합니다.

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

# app = Dash(__name__)
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

Dash is running on http://127.0.0.1:8050/



## 여러 콜백 수행하기

콜백함수에 여러개의 입력과 출력을 사용할 수 있다. @app.callback 데코레이터 내에 정의한 순서만 맞춰주자.

html의 form과 비슷한 패턴으로 입력을 받아야하는 경우가 있는데 이때는 콜백함수에 해당하는 모든 인풋을 넣는 것으로 해결 가능하다. 

In [5]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

# app = Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        dcc.Input(id="input-1", type="text", value="Montréal"),
        dcc.Input(id="input-2", type="text", value="Canada"),
        html.Div(id="number-output"),
    ]
)


@app.callback(
    Output("number-output", "children"),
    Input("input-1", "value"),
    Input("input-2", "value"),
)
def update_output(input1, input2):
    return u'Input 1 is "{}" and Input 2 is "{}"'.format(input1, input2)


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')

Dash is running on http://127.0.0.1:8050/

